In [1]:
import json
from pennylane import numpy as np
import pennylane as qml
import scipy

In [2]:
# Uneditable section #

def abs_dist(rho, sigma):
    """A function to compute the absolute value |rho - sigma|."""
    polar = scipy.linalg.polar(rho - sigma)
    return polar[1]

def word_dist(word):
    """A function which counts the non-identity operators in a Pauli word"""
    return sum(word[i] != "I" for i in range(len(word)))


# Produce the Pauli density for a given Pauli word and apply noise

def noisy_Pauli_density(word, lmbda):
    """
       A subcircuit which prepares a density matrix (I + P)/2**n for a given Pauli
       word P, and applies depolarizing noise to each qubit. Nothing is returned.

    Args:
            word (str): A Pauli word represented as a string with characters I,  X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.
    """

    # End of uneditable section #

    
    # Put your code here #
    
    n_qubits = len(word)
    
    word_dict = {i:pauli for i,pauli in enumerate(word)}
    iden_dict = {i:"I" for i in range(n_qubits)}
    
    pw1 = qml.pauli.PauliWord(iden_dict)
    pw2 = qml.pauli.PauliWord(word_dict)
    ps = qml.pauli.PauliSentence({pw1: 1/2**n_qubits, pw2: 1/2**n_qubits})
    
    qml.QubitDensityMatrix(ps.to_mat(wire_order = range(n_qubits)), wires=range(n_qubits))
    
    for wire in range(n_qubits):
        qml.DepolarizingChannel(lmbda, wires=wire)
    
    


# Uneditable section #
        
# Compute the trace distance from a noisy Pauli density to the maximally mixed density

def maxmix_trace_dist(word, lmbda):
    """
       A function compute the trace distance between a noisy density matrix, specified
       by a Pauli word, and the maximally mixed matrix.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The trace distance between two matrices encoding Pauli words.
    """

    # End of uneditable section #


    # Put your code here #
    n_qubits = len(word)
    dev = qml.device('default.mixed', wires=n_qubits)
    @qml.qnode(dev)
    def rho():
        noisy_Pauli_density(word, lmbda)
        return qml.density_matrix(range(n_qubits))
    
    rho = rho()
    sigma = 1/2**n_qubits*np.eye(2**n_qubits)
    
    return 1/2*np.trace(abs_dist(rho, sigma))


# Uneditable section #

def bound_verifier(word, lmbda):
    """
       A simple check function which verifies the trace distance from a noisy Pauli density
       to the maximally mixed matrix is bounded by (1 - lambda)^|P|.

    Args:
            word (str): A Pauli word represented as a string with characters I, X, Y and Z.
            lmbda (float): The probability of replacing a qubit with something random.

    Returns:
            float: The difference between (1 - lambda)^|P| and T(rho_P(lambda), rho_0).
    """

    # End of uneditable section #

    
    # Put your code here #
      
    return (1-lmbda)**word_dist(word) - maxmix_trace_dist(word, lmbda)

In [3]:
def run(test_case_input: str) -> str:

    word, lmbda = json.loads(test_case_input)
    output = np.real(bound_verifier(word, lmbda))

    return str(output)


def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your trace distance isn't quite right!"

In [4]:
test_cases = [['["XXI", 0.7]', '0.0877777777777777'], ['["XXIZ", 0.1]', '0.4035185185185055'], ['["YIZ", 0.3]', '0.30999999999999284'], ['["ZZZZZZZXXX", 0.1]', '0.22914458207245006'], ['["XZIXZIYY", 0.8]', '6.395610425240046e-05'], ['["ZZZZZZZZZZ", 0.1]', '0.22914458207245003']]

In [5]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '["XXI", 0.7]'...
Correct!
Running test case 1 with input '["XXIZ", 0.1]'...
Correct!
Running test case 2 with input '["YIZ", 0.3]'...
Correct!
Running test case 3 with input '["ZZZZZZZXXX", 0.1]'...
Correct!
Running test case 4 with input '["XZIXZIYY", 0.8]'...
Correct!
Running test case 5 with input '["ZZZZZZZZZZ", 0.1]'...
Correct!
